<a href="https://colab.research.google.com/github/SamuelW669/DeBaussyAI/blob/main/Accuracy_loss_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from music21 import converter

# Load and parse the MusicXML file
score = converter.parse('path/to/your/file.musicxml')

# Extract the notes and their attributes
notes = []
for part in score.parts:
    for note in part.notes:
        notes.append({
            'pitch': note.pitch.midi,  # MIDI number
            'start_time': note.offset,  # Start time in quarter lengths
            'duration': note.quarterLength  # Duration in quarter lengths
        })


FileNotFoundError: Cannot find file in path/to/your/file.musicxml

In [ ]:
from pydub import AudioSegment

# Load the MP3 file
audio = AudioSegment.from_mp3('path/to/your/file.mp3')

# Export as WAV
audio.export('file.wav', format='wav')


ModuleNotFoundError: No module named 'pydub'

In [ ]:
import librosa
import numpy as np

# Load the WAV file
y, sr = librosa.load('file.wav')

# Extract pitch (intonation)
pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)

# Extract onset times (rhythm)
onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
onset_times = librosa.frames_to_time(onset_frames, sr=sr)


In [ ]:
def compare_intonation(expected_notes, detected_pitches):
    # Compare expected and detected pitches
    intonation_results = []
    for i, note in enumerate(expected_notes):
        detected_pitch = detected_pitches[i]
        if abs(note['pitch'] - detected_pitch) < some_threshold:
            intonation_results.append(True)
        else:
            intonation_results.append(False)
    return intonation_results

def compare_rhythm(expected_notes, onset_times):
    # Compare expected and detected onset times
    rhythm_results = []
    for i, note in enumerate(expected_notes):
        detected_onset = onset_times[i]
        if abs(note['start_time'] - detected_onset) < some_threshold:
            rhythm_results.append(True)
        else:
            rhythm_results.append(False)
    return rhythm_results

# Assume `detected_pitches` and `onset_times` are obtained from librosa analysis
intonation_results = compare_intonation(notes, detected_pitches)
rhythm_results = compare_rhythm(notes, onset_times)

# Display the results
print("Intonation results:", intonation_results)
print("Rhythm results:", rhythm_results)


In [ ]:
pip install music21 librosa pydub fastdtw numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512614 sha256=9faede1e4a3881ee1845a91b08f7475d84c70714a4f27d8941075875e3254fa2
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw


In [ ]:
from music21 import converter
from pydub import AudioSegment
import librosa
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Step 1: Parse the .musicxml file
def parse_musicxml(file_path):
    score = converter.parse(file_path)
    notes = []
    for part in score.parts:
        for note in part.notes:
            notes.append({
                'pitch': note.pitch.midi,  # MIDI number
                'start_time': note.offset,  # Start time in quarter lengths
                'duration': note.quarterLength  # Duration in quarter lengths
            })
    return notes

# Step 2: Convert MP3 to WAV
def convert_mp3_to_wav(mp3_path, wav_path):
    audio = AudioSegment.from_mp3(mp3_path)
    audio.export(wav_path, format='wav')

# Step 3: Analyze the WAV file
def analyze_wav(wav_path):
    y, sr = librosa.load(wav_path)
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
    onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
    onset_times = librosa.frames_to_time(onset_frames, sr=sr)

    # Extract the pitches at onset times
    detected_pitches = []
    for onset_time in onset_times:
        frame = librosa.time_to_frames(onset_time, sr=sr)
        pitch = pitches[:, frame].max()
        if pitch > 0:  # Ignore zero pitch values
            detected_pitches.append((pitch, onset_time))

    return detected_pitches

# Step 4: Align detected pitches with expected notes
def align_notes(expected_notes, detected_pitches):
    expected_pitches = np.array([note['pitch'] for note in expected_notes])
    expected_times = np.array([note['start_time'] for note in expected_notes])

    detected_pitches_array = np.array([pitch for pitch, time in detected_pitches])
    detected_times = np.array([time for pitch, time in detected_pitches])

    distance, path = fastdtw(expected_pitches, detected_pitches_array, dist=euclidean)

    aligned_pitches = []
    for i, j in path:
        aligned_pitches.append((expected_times[i], expected_pitches[i], detected_times[j], detected_pitches_array[j]))

    return aligned_pitches

# Step 5: Compare intonation and rhythm
def compare_intonation_and_rhythm(aligned_pitches, intonation_threshold=1.0, rhythm_threshold=0.1): # Thresholds control the tolerance
    results = []
    for expected_time, expected_pitch, detected_time, detected_pitch in aligned_pitches:
        intonation_correct = abs(expected_pitch - detected_pitch) < intonation_threshold
        rhythm_correct = abs(expected_time - detected_time) < rhythm_threshold
        results.append((expected_time, expected_pitch, detected_time, detected_pitch, intonation_correct, rhythm_correct))
    return results

# Main program
def main():
    musicxml_path = 'path/to/your/file.musicxml'
    mp3_path = 'path/to/your/file.mp3'
    wav_path = 'path/to/your/file.wav'

    # Parse MusicXML
    expected_notes = parse_musicxml(musicxml_path)

    # Convert MP3 to WAV
    convert_mp3_to_wav(mp3_path, wav_path)

    # Analyze WAV
    detected_pitches = analyze_wav(wav_path)

    # Align detected pitches with expected notes
    aligned_pitches = align_notes(expected_notes, detected_pitches)

    # Compare intonation and rhythm
    results = compare_intonation_and_rhythm(aligned_pitches)

    # Print the results
    for result in results:
        expected_time, expected_pitch, detected_time, detected_pitch, intonation_correct, rhythm_correct = result
        print(f"Expected: {expected_pitch} at {expected_time}s, Detected: {detected_pitch} at {detected_time}s")
        print(f"Intonation Correct: {intonation_correct}, Rhythm Correct: {rhythm_correct}")

if __name__ == "__main__":
    main()


In [ ]:
from music21 import converter
from pydub import AudioSegment
import librosa
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Parse the .musicxml file
def parse_musicxml(file_path):
    score = converter.parse(file_path)
    notes = []
    for part in score.parts:
        for note in part.notes:
            notes.append({
                'pitch': note.pitch.midi,
                'start_time': note.offset,
                'duration': note.quarterLength
            })
    return notes

# Convert MP3 to WAV
def convert_mp3_to_wav(mp3_path, wav_path):
    audio = AudioSegment.from_mp3(mp3_path)
    audio.export(wav_path, format='wav')

# Detect vibrato by identifying rapid pitch changes
def detect_vibrato(pitches, threshold=1.0):
    vibrato_indices = []
    for i in range(1, len(pitches)):
        if abs(pitches[i] - pitches[i - 1]) > threshold:
            vibrato_indices.append(i)
    return vibrato_indices

# Smooth pitches by averaging over a window around vibrato indices
def smooth_pitches(pitches, vibrato_indices, window_size=5):
    smoothed_pitches = np.copy(pitches)
    for i in vibrato_indices:
        start = max(0, i - window_size // 2)
        end = min(len(pitches), i + window_size // 2 + 1)
        window = pitches[start:end]
        smoothed_pitches[i] = np.mean(window)
    return smoothed_pitches

# Analyze the WAV file
def analyze_wav(wav_path):
    y, sr = librosa.load(wav_path)
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
    onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
    onset_times = librosa.frames_to_time(onset_frames, sr=sr)

    # Extract the pitches at onset times
    detected_pitches = []
    for onset_time in onset_times:
        frame = librosa.time_to_frames(onset_time, sr=sr)
        pitch = pitches[:, frame].max()
        if pitch > 0:
            detected_pitches.append((pitch, onset_time))

    # Separate pitch values and times
    pitch_values = [pitch for pitch, time in detected_pitches]
    vibrato_indices = detect_vibrato(pitch_values)

    # Smooth the detected pitches to handle vibrato
    smoothed_pitch_values = smooth_pitches(pitch_values, vibrato_indices)

    # Reconstruct detected_pitches with smoothed values
    detected_pitches = [(smoothed_pitch_values[i], detected_pitches[i][1]) for i in range(len(detected_pitches))]

    return detected_pitches

# Align detected pitches with expected notes
def align_notes(expected_notes, detected_pitches):
    expected_pitches = np.array([note['pitch'] for note in expected_notes])
    expected_times = np.array([note['start_time'] for note in expected_notes])

    detected_pitches_array = np.array([pitch for pitch, time in detected_pitches])
    detected_times = np.array([time for pitch, time in detected_pitches])

    distance, path = fastdtw(expected_pitches, detected_pitches_array, dist=euclidean)

    aligned_pitches = []
    for i, j in path:
        aligned_pitches.append((expected_times[i], expected_pitches[i], detected_times[j], detected_pitches_array[j]))

    return aligned_pitches

# Compare intonation and rhythm
def compare_intonation_and_rhythm(aligned_pitches, intonation_threshold=1.0, rhythm_threshold=0.1):
    results = []
    for expected_time, expected_pitch, detected_time, detected_pitch in aligned_pitches:
        intonation_correct = abs(expected_pitch - detected_pitch) < intonation_threshold
        rhythm_correct = abs(expected_time - detected_time) < rhythm_threshold
        results.append((expected_time, expected_pitch, detected_time, detected_pitch, intonation_correct, rhythm_correct))
    return results

# Main program
def main():
    musicxml_path = 'path/to/your/file.musicxml'
    mp3_path = 'path/to/your/file.mp3'
    wav_path = 'path/to/your/file.wav'

    # Parse MusicXML
    expected_notes = parse_musicxml(musicxml_path)

    # Convert MP3 to WAV
    convert_mp3_to_wav(mp3_path, wav_path)

    # Analyze WAV
    detected_pitches = analyze_wav(wav_path)

    # Align detected pitches with expected notes
    aligned_pitches = align_notes(expected_notes, detected_pitches)

    # Compare intonation and rhythm with adjusted tolerances
    results = compare_intonation_and_rhythm(aligned_pitches, intonation_threshold=0.5, rhythm_threshold=0.05)

    # Print the results
    for result in results:
        expected_time, expected_pitch, detected_time, detected_pitch, intonation_correct, rhythm_correct = result
        print(f"Expected: {expected_pitch} at {expected_time}s, Detected: {detected_pitch} at {detected_time}s")
        print(f"Intonation Correct: {intonation_correct}, Rhythm Correct: {rhythm_correct}")

if __name__ == "__main__":
    main()


In [ ]:
from music21 import converter
from pydub import AudioSegment
import librosa
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Parse the .musicxml file
def parse_musicxml(file_path):
    score = converter.parse(file_path)
    notes = []
    for part in score.parts:
        for note in part.notes:
            notes.append({
                'pitch': note.pitch.midi,
                'start_time': note.offset,
                'duration': note.quarterLength
            })
    return notes

# Convert MP3 to WAV
def convert_mp3_to_wav(mp3_path, wav_path):
    audio = AudioSegment.from_mp3(mp3_path)
    audio.export(wav_path, format='wav')

# Detect vibrato by identifying rapid pitch changes
def detect_vibrato(pitches, threshold=1.0):
    vibrato_indices = []
    for i in range(1, len(pitches)):
        if abs(pitches[i] - pitches[i - 1]) > threshold:
            vibrato_indices.append(i)
    return vibrato_indices

# Smooth pitches by averaging over a window around vibrato indices
def smooth_pitches(pitches, vibrato_indices, window_size=5):
    smoothed_pitches = np.copy(pitches)
    half_window = window_size // 2

    for i in vibrato_indices:
        start = max(0, i - half_window)
        end = min(len(pitches), i + half_window + 1)
        window = pitches[start:end]
        smoothed_pitches[i] = np.mean(window)

    return smoothed_pitches

# Analyze the WAV file
def analyze_wav(wav_path):
    y, sr = librosa.load(wav_path)
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
    onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
    onset_times = librosa.frames_to_time(onset_frames, sr=sr)

    # Extract the pitches at onset times
    detected_pitches = []
    for onset_time in onset_times:
        frame = librosa.time_to_frames(onset_time, sr=sr)
        pitch = pitches[:, frame].max()
        if pitch > 0:
            detected_pitches.append((pitch, onset_time))

    # Separate pitch values and times
    pitch_values = [pitch for pitch, time in detected_pitches]
    vibrato_indices = detect_vibrato(pitch_values)

    # Smooth the detected pitches to handle vibrato
    smoothed_pitch_values = smooth_pitches(pitch_values, vibrato_indices)

    # Reconstruct detected_pitches with smoothed values
    detected_pitches = [(smoothed_pitch_values[i], detected_pitches[i][1]) for i in range(len(detected_pitches))]

    return detected_pitches

# Align detected pitches with expected notes
def align_notes(expected_notes, detected_pitches):
    expected_pitches = np.array([note['pitch'] for note in expected_notes])
    expected_times = np.array([note['start_time'] for note in expected_notes])

    detected_pitches_array = np.array([pitch for pitch, time in detected_pitches])
    detected_times = np.array([time for pitch, time in detected_pitches])

    distance, path = fastdtw(expected_pitches, detected_pitches_array, dist=euclidean)

    aligned_pitches = []
    for i, j in path:
        aligned_pitches.append((expected_times[i], expected_pitches[i], detected_times[j], detected_pitches_array[j]))

    return aligned_pitches

# Compare intonation and rhythm
def compare_intonation_and_rhythm(aligned_pitches, intonation_threshold=1.0, rhythm_threshold=0.1):
    results = []
    for expected_time, expected_pitch, detected_time, detected_pitch in aligned_pitches:
        intonation_correct = abs(expected_pitch - detected_pitch) < intonation_threshold
        rhythm_correct = abs(expected_time - detected_time) < rhythm_threshold
        results.append((expected_time, expected_pitch, detected_time, detected_pitch, intonation_correct, rhythm_correct))
    return results

# Main program
def main():
    musicxml_path = 'path/to/your/file.musicxml'
    mp3_path = 'path/to/your/file.mp3'
    wav_path = 'path/to/your/file.wav'

    # Parse MusicXML
    expected_notes = parse_musicxml(musicxml_path)

    # Convert MP3 to WAV
    convert_mp3_to_wav(mp3_path, wav_path)

    # Analyze WAV
    detected_pitches = analyze_wav(wav_path)

    # Align detected pitches with expected notes
    aligned_pitches = align_notes(expected_notes, detected_pitches)

    # Compare intonation and rhythm with adjusted tolerances
    results = compare_intonation_and_rhythm(aligned_pitches, intonation_threshold=0.5, rhythm_threshold=0.05)

    # Print the results
    for result in results:
        expected_time, expected_pitch, detected_time, detected_pitch, intonation_correct, rhythm_correct = result
        print(f"Expected: {expected_pitch} at {expected_time}s, Detected: {detected_pitch} at {detected_time}s")
        print(f"Intonation Correct: {intonation_correct}, Rhythm Correct: {rhythm_correct}")

if __name__ == "__main__":
    main()
